In [92]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from textblob import TextBlob
import urllib.parse
import time
import os
from dotenv import load_dotenv
import re

from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch 
import torch.nn.functional as F
from tqdm import tqdm

In [ ]:
load_dotenv()
api_key = os.getenv('News_API_KEY')
if not api_key:
    raise ValueError("API key is missing. Set the News_API_KEY environment variable in the .env file.")
tickers = ["Apple", "Google", "Microsoft", "Amazon", "Meta", "Nvidia", "Tesla", "Netflix", "AMD", "Palantir"]
base_url = f"https://newsapi.org/v2/everything?"
sentiment_data = []
news_data = []
call_count = 0

In [ ]:
for i in range(5):
    date = datetime.utcnow() - timedelta(days = i)
    from_date = to_date = date.strftime("%Y-%m-%d")

    for ticker in tickers:
        print(f"Pulling news for {ticker} on {from_date}")
        
        params = {
            "q": f'"{ticker}"',
            "searchIn": "title",
            "from": from_date,
            "to": to_date,
            "language": "en",
            "sortBy": "relevancy",
            "pageSize": 10,
            "apiKey": api_key
        }
        try:
            r = requests.get(base_url, params = params)
            r.raise_for_status()
            articles = r.json().get("articles", [])
        
        except Exception as e:
            print(f"Error fetching data from {ticker} on {from_date}: {e}")
            continue
            
        for article in articles:
            article["ticker"] = ticker
            # getting first 10 character of the timestamp string yyyy-mm-dd
            article["date"] = article.get("publishedAt", "")[:10] 
            news_data.append(article)

        time.sleep(1.1)

/var/folders/54/hwsm49bs033bpdy18h8fnp2h0000gn/T/ipykernel_66464/1101442247.py:2: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  date = datetime.utcnow() - timedelta(days = i)


Pulling news for Apple on 2025-05-07
Pulling news for Google on 2025-05-07
Pulling news for Microsoft on 2025-05-07
Pulling news for Amazon on 2025-05-07
Pulling news for Meta on 2025-05-07
Pulling news for Nvidia on 2025-05-07
Pulling news for Tesla on 2025-05-07
Pulling news for Netflix on 2025-05-07
Pulling news for AMD on 2025-05-07
Pulling news for Palantir on 2025-05-07
Pulling news for Apple on 2025-05-06
Pulling news for Google on 2025-05-06
Pulling news for Microsoft on 2025-05-06
Pulling news for Amazon on 2025-05-06
Pulling news for Meta on 2025-05-06
Pulling news for Nvidia on 2025-05-06
Pulling news for Tesla on 2025-05-06
Pulling news for Netflix on 2025-05-06
Pulling news for AMD on 2025-05-06
Pulling news for Palantir on 2025-05-06
Pulling news for Apple on 2025-05-05
Pulling news for Google on 2025-05-05
Pulling news for Microsoft on 2025-05-05
Pulling news for Amazon on 2025-05-05
Pulling news for Meta on 2025-05-05
Pulling news for Nvidia on 2025-05-05
Pulling news f

In [128]:
news_df = pd.DataFrame(news_data)
news_df

,source,author,title,description,url,urlToImage,publishedAt,content,ticker,date
0,"{'id': 'wired', 'name': 'Wired'}",Kate Knibbs,Trump’s Tariffs Are Threatening America’s Appl...,Most of the apple juice Americans drink is imp...,https://www.wired.com/story/apple-juice-shorta...,https://media.wired.com/photos/6812934a8f16f41...,2025-05-06T11:00:00Z,"Few foods are more American than apple pie, bu...",Apple,2025-05-06
1,"{'id': None, 'name': 'MacRumors'}",Joe Rossignol,Apple Says These Five Games Are Coming to Appl...,Apple today announced that five additional gam...,https://www.macrumors.com/2025/05/06/apple-arc...,https://images.macrumors.com/t/qeSA2lNWJQSsmfC...,2025-05-06T14:25:41Z,Apple today announced that five additional gam...,Apple,2025-05-06
2,"{'id': None, 'name': 'MacRumors'}",Joe Rossignol,Two Highly-Rated Apple TV+ Comedy Series Renewed,Apple today announced it has renewed two accla...,https://www.macrumors.com/2025/05/06/two-apple...,https://images.macrumors.com/t/yvromLmrtBgWK_N...,2025-05-06T20:46:38Z,Apple todayannounced it has renewed two acclai...,Apple,2025-05-06
3,"{'id': None, 'name': 'MacRumors'}",Tim Hardwick,Apple Supplier Plans Second Indian Facility fo...,Apple supplier Jabil is planning to expand its...,https://www.macrumors.com/2025/05/06/apple-sup...,https://images.macrumors.com/t/2HTpELVLXLYZFGe...,2025-05-06T10:31:04Z,Apple supplier Jabil is planning to expand its...,Apple,2025-05-06
4,"{'id': None, 'name': 'Android Central'}",nickodiaz@sbcglobal.net (Nickolas Diaz),Google teases Apple for stealing its design in...,Google's Pixel teased Apple over its rumored i...,https://www.androidcentral.com/phones/google-p...,https://cdn.mos.cms.futurecdn.net/fh74GxgsiaGa...,2025-05-06T18:28:05Z,What you need to know\r\n<ul><li>Google's Pixe...,Apple,2025-05-06
...,...,...,...,...,...,...,...,...,...,...
377,"{'id': None, 'name': 'Biztoc.com'}",finance.yahoo.com,"Advanced Micro Devices, Inc. (AMD): Among Ken ...",We recently published an article titled Billio...,https://biztoc.com/x/182ca59718f1bf27,https://biztoc.com/cdn/182ca59718f1bf27_s.webp,2025-05-03T14:54:43Z,We recently published an article titled Billio...,AMD,2025-05-03
378,"{'id': None, 'name': 'Yahoo Entertainment'}",None,Prediction: 2 Stocks Will Be Worth More Than P...,None,https://consent.yahoo.com/v2/collectConsent?se...,None,2025-05-03T08:04:00Z,"If you click 'Accept all', we and our partners...",Palantir,2025-05-03
379,"{'id': None, 'name': 'Biztoc.com'}",finance.yahoo.com,Prediction: This Super Software Growth Stock W...,In This Article:\nKey Points\n-\nArtificial in...,https://biztoc.com/x/752a6e69a4a8e12d,https://biztoc.com/cdn/752a6e69a4a8e12d_s.webp,2025-05-03T13:27:02Z,In This Article:Key Points-Artificial intellig...,Palantir,2025-05-03
380,"{'id': None, 'name': 'Biztoc.com'}",npr.org,"How Palantir, the secretive tech company, is r...","How Palantir, the secretive tech company, is r...",https://biztoc.com/x/fd03bb83c573534c,https://biztoc.com/cdn/fd03bb83c573534c_s.webp,2025-05-03T12:43:10Z,"How Palantir, the secretive tech company, is r...",Palantir,2025-05-03


In [129]:
# Only apply this cleanup to 'Apple' ticker
mask = news_df['ticker'] == 'Apple'

fruit_keywords = ['fruit', 'juice', 'pie', 'orchard', 'cider', 'farming', 'produce']

def is_apple_company(row):
    text = (str(row['title']) + ' ' + str(row['description'])).lower()
    return not any(re.search(rf'\b{kw}\b', text) for kw in fruit_keywords)

dropped = news_df[mask].loc[~news_df[mask].apply(is_apple_company, axis=1)]
print("Filtered Apple rows:")
print(dropped[['title', 'description']])

# Filter only Apple rows with the function, keep the rest untouched
news_df = pd.concat([
    news_df[mask].loc[news_df[mask].apply(is_apple_company, axis=1)], # filtered Apple articles
    news_df[~mask]  # non-Apple articles
], ignore_index=True)

Filtered Apple rows:
                                               title  \
0  Trump’s Tariffs Are Threatening America’s Appl...   

                                         description  
0  Most of the apple juice Americans drink is imp...  


In [130]:
news_df

,source,author,title,description,url,urlToImage,publishedAt,content,ticker,date
0,"{'id': None, 'name': 'MacRumors'}",Joe Rossignol,Apple Says These Five Games Are Coming to Appl...,Apple today announced that five additional gam...,https://www.macrumors.com/2025/05/06/apple-arc...,https://images.macrumors.com/t/qeSA2lNWJQSsmfC...,2025-05-06T14:25:41Z,Apple today announced that five additional gam...,Apple,2025-05-06
1,"{'id': None, 'name': 'MacRumors'}",Joe Rossignol,Two Highly-Rated Apple TV+ Comedy Series Renewed,Apple today announced it has renewed two accla...,https://www.macrumors.com/2025/05/06/two-apple...,https://images.macrumors.com/t/yvromLmrtBgWK_N...,2025-05-06T20:46:38Z,Apple todayannounced it has renewed two acclai...,Apple,2025-05-06
2,"{'id': None, 'name': 'MacRumors'}",Tim Hardwick,Apple Supplier Plans Second Indian Facility fo...,Apple supplier Jabil is planning to expand its...,https://www.macrumors.com/2025/05/06/apple-sup...,https://images.macrumors.com/t/2HTpELVLXLYZFGe...,2025-05-06T10:31:04Z,Apple supplier Jabil is planning to expand its...,Apple,2025-05-06
3,"{'id': None, 'name': 'Android Central'}",nickodiaz@sbcglobal.net (Nickolas Diaz),Google teases Apple for stealing its design in...,Google's Pixel teased Apple over its rumored i...,https://www.androidcentral.com/phones/google-p...,https://cdn.mos.cms.futurecdn.net/fh74GxgsiaGa...,2025-05-06T18:28:05Z,What you need to know\r\n<ul><li>Google's Pixe...,Apple,2025-05-06
4,"{'id': None, 'name': 'CNET'}",Omar Gallaga,Developers Sue Apple for Failing to Comply Wit...,A class action suit alleges that Apple undercu...,https://www.cnet.com/tech/services-and-softwar...,https://www.cnet.com/a/img/resize/ae1fe460325b...,2025-05-06T15:41:00Z,A group of app developers have filed a class a...,Apple,2025-05-06
...,...,...,...,...,...,...,...,...,...,...
376,"{'id': None, 'name': 'Biztoc.com'}",finance.yahoo.com,"Advanced Micro Devices, Inc. (AMD): Among Ken ...",We recently published an article titled Billio...,https://biztoc.com/x/182ca59718f1bf27,https://biztoc.com/cdn/182ca59718f1bf27_s.webp,2025-05-03T14:54:43Z,We recently published an article titled Billio...,AMD,2025-05-03
377,"{'id': None, 'name': 'Yahoo Entertainment'}",None,Prediction: 2 Stocks Will Be Worth More Than P...,None,https://consent.yahoo.com/v2/collectConsent?se...,None,2025-05-03T08:04:00Z,"If you click 'Accept all', we and our partners...",Palantir,2025-05-03
378,"{'id': None, 'name': 'Biztoc.com'}",finance.yahoo.com,Prediction: This Super Software Growth Stock W...,In This Article:\nKey Points\n-\nArtificial in...,https://biztoc.com/x/752a6e69a4a8e12d,https://biztoc.com/cdn/752a6e69a4a8e12d_s.webp,2025-05-03T13:27:02Z,In This Article:Key Points-Artificial intellig...,Palantir,2025-05-03
379,"{'id': None, 'name': 'Biztoc.com'}",npr.org,"How Palantir, the secretive tech company, is r...","How Palantir, the secretive tech company, is r...",https://biztoc.com/x/fd03bb83c573534c,https://biztoc.com/cdn/fd03bb83c573534c_s.webp,2025-05-03T12:43:10Z,"How Palantir, the secretive tech company, is r...",Palantir,2025-05-03


In [131]:
news_df['source'] = news_df['source'].apply(lambda x: x.get('name') if isinstance(x, dict) else None)

In [132]:
news_df = news_df[['ticker', 'date', 'source', 'title',	'description', 'url', 'content']]

In [133]:
news_df

,ticker,date,source,title,description,url,content
0,Apple,2025-05-06,MacRumors,Apple Says These Five Games Are Coming to Appl...,Apple today announced that five additional gam...,https://www.macrumors.com/2025/05/06/apple-arc...,Apple today announced that five additional gam...
1,Apple,2025-05-06,MacRumors,Two Highly-Rated Apple TV+ Comedy Series Renewed,Apple today announced it has renewed two accla...,https://www.macrumors.com/2025/05/06/two-apple...,Apple todayannounced it has renewed two acclai...
2,Apple,2025-05-06,MacRumors,Apple Supplier Plans Second Indian Facility fo...,Apple supplier Jabil is planning to expand its...,https://www.macrumors.com/2025/05/06/apple-sup...,Apple supplier Jabil is planning to expand its...
3,Apple,2025-05-06,Android Central,Google teases Apple for stealing its design in...,Google's Pixel teased Apple over its rumored i...,https://www.androidcentral.com/phones/google-p...,What you need to know\r\n<ul><li>Google's Pixe...
4,Apple,2025-05-06,CNET,Developers Sue Apple for Failing to Comply Wit...,A class action suit alleges that Apple undercu...,https://www.cnet.com/tech/services-and-softwar...,A group of app developers have filed a class a...
...,...,...,...,...,...,...,...
376,AMD,2025-05-03,Biztoc.com,"Advanced Micro Devices, Inc. (AMD): Among Ken ...",We recently published an article titled Billio...,https://biztoc.com/x/182ca59718f1bf27,We recently published an article titled Billio...
377,Palantir,2025-05-03,Yahoo Entertainment,Prediction: 2 Stocks Will Be Worth More Than P...,None,https://consent.yahoo.com/v2/collectConsent?se...,"If you click 'Accept all', we and our partners..."
378,Palantir,2025-05-03,Biztoc.com,Prediction: This Super Software Growth Stock W...,In This Article:\nKey Points\n-\nArtificial in...,https://biztoc.com/x/752a6e69a4a8e12d,In This Article:Key Points-Artificial intellig...
379,Palantir,2025-05-03,Biztoc.com,"How Palantir, the secretive tech company, is r...","How Palantir, the secretive tech company, is r...",https://biztoc.com/x/fd03bb83c573534c,"How Palantir, the secretive tech company, is r..."


In [134]:
news_df = news_df.dropna(subset = ['description'])

In [137]:
#news_df = pd.DataFrame(news_data)
print(f'The size of the dataframe: {len(news_df)}')
news_df.isna().sum()

The size of the dataframe: 328


ticker         0
date           0
source         0
title          0
description    0
url            0
content        0
dtype: int64

### Sentiment Analysis using FinBERT

In [138]:
# Load FinBERT model
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")
finbert_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device = 0)

In [139]:
def get_finbert_scores_batch(text_list):
    scores = []

    for text in text_list:
        if not text or not text.strip():
            scores.append(None)
            continue

        try:
            # Tokenize with truncation
            inputs = tokenizer(
                text,
                return_tensors="pt",
                truncation=True,
                max_length=512,
                padding=True
            )

            # Force CPU (avoid MPS crash)
            inputs = {k: v.cpu() for k, v in inputs.items()}
            model.cpu()

            with torch.no_grad():
                outputs = model(**inputs)
                logits = outputs.logits
                probs = F.softmax(logits, dim=1).squeeze()

                # Weighted score: +1*pos, -1*neg, 0*neutral
                score = probs[0].item() * 1 + probs[1].item() * -1
                scores.append(score)

        except Exception as e:
            print(f"Error on text: {e}")
            scores.append(None)

    return scores


In [140]:
news_df['text'] = news_df['title'].fillna('') + ' ' + news_df['description'].fillna('')
sentiments = []

# Batch size: adjust lower if crash continues
batch_size = 16
text_list = news_df['text'].tolist()

for i in tqdm(range(0, len(text_list), batch_size)):
    batch = text_list[i:i+batch_size]
    scores = get_finbert_scores_batch(batch)
    sentiments.extend(scores)

news_df['sentiment'] = sentiments

/var/folders/54/hwsm49bs033bpdy18h8fnp2h0000gn/T/ipykernel_66464/4167040519.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df['text'] = news_df['title'].fillna('') + ' ' + news_df['description'].fillna('')
100%|██████████| 21/21 [00:17<00:00,  1.21it/s]
/var/folders/54/hwsm49bs033bpdy18h8fnp2h0000gn/T/ipykernel_66464/4167040519.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df['sentiment'] = sentiments


In [156]:
news_df

,ticker,date,source,title,description,url,content,text,sentiment
0,Apple,2025-05-06,MacRumors,Apple Says These Five Games Are Coming to Appl...,Apple today announced that five additional gam...,https://www.macrumors.com/2025/05/06/apple-arc...,Apple today announced that five additional gam...,Apple Says These Five Games Are Coming to Appl...,0.997901
1,Apple,2025-05-06,MacRumors,Two Highly-Rated Apple TV+ Comedy Series Renewed,Apple today announced it has renewed two accla...,https://www.macrumors.com/2025/05/06/two-apple...,Apple todayannounced it has renewed two acclai...,Two Highly-Rated Apple TV+ Comedy Series Renew...,0.971871
2,Apple,2025-05-06,MacRumors,Apple Supplier Plans Second Indian Facility fo...,Apple supplier Jabil is planning to expand its...,https://www.macrumors.com/2025/05/06/apple-sup...,Apple supplier Jabil is planning to expand its...,Apple Supplier Plans Second Indian Facility fo...,0.999958
3,Apple,2025-05-06,Android Central,Google teases Apple for stealing its design in...,Google's Pixel teased Apple over its rumored i...,https://www.androidcentral.com/phones/google-p...,What you need to know\r\n<ul><li>Google's Pixe...,Google teases Apple for stealing its design in...,0.996715
4,Apple,2025-05-06,CNET,Developers Sue Apple for Failing to Comply Wit...,A class action suit alleges that Apple undercu...,https://www.cnet.com/tech/services-and-softwar...,A group of app developers have filed a class a...,Developers Sue Apple for Failing to Comply Wit...,-0.007718
...,...,...,...,...,...,...,...,...,...
375,AMD,2025-05-03,Biztoc.com,If You Invested $10K In AMD Stock 10 Years Ago...,In This Article:\nBenzinga and Yahoo Finance L...,https://biztoc.com/x/c6200495c08fb13c,In This Article:Benzinga and Yahoo Finance LLC...,If You Invested $10K In AMD Stock 10 Years Ago...,0.999989
376,AMD,2025-05-03,Biztoc.com,"Advanced Micro Devices, Inc. (AMD): Among Ken ...",We recently published an article titled Billio...,https://biztoc.com/x/182ca59718f1bf27,We recently published an article titled Billio...,"Advanced Micro Devices, Inc. (AMD): Among Ken ...",-0.998734
378,Palantir,2025-05-03,Biztoc.com,Prediction: This Super Software Growth Stock W...,In This Article:\nKey Points\n-\nArtificial in...,https://biztoc.com/x/752a6e69a4a8e12d,In This Article:Key Points-Artificial intellig...,Prediction: This Super Software Growth Stock W...,-0.999998
379,Palantir,2025-05-03,Biztoc.com,"How Palantir, the secretive tech company, is r...","How Palantir, the secretive tech company, is r...",https://biztoc.com/x/fd03bb83c573534c,"How Palantir, the secretive tech company, is r...","How Palantir, the secretive tech company, is r...",-0.648749


In [ ]:
news_df.to_csv("assets/news_data.csv", index = False)

In [164]:
sentiment_data = []
grouped = news_df.groupby(['ticker', 'date'])
for (ticker, date), group in grouped:
    article_count = group['sentiment'].count()
    avg_sentiment = group['sentiment'].mean()

    sentiment_data.append({
        "ticker": ticker,
        "date": date,
        "article_count": article_count,
        "avg_sentiment": avg_sentiment
    })

In [173]:
sentiment_df = pd.DataFrame(sentiment_data)
sentiment_df

,ticker,date,article_count,avg_sentiment
0,AMD,2025-05-03,8,0.617277
1,AMD,2025-05-04,9,0.112336
2,AMD,2025-05-05,9,0.193812
3,AMD,2025-05-06,9,-0.152049
4,Amazon,2025-05-03,10,0.509995
5,Amazon,2025-05-04,10,0.591176
6,Amazon,2025-05-05,10,0.712749
7,Amazon,2025-05-06,10,0.868796
8,Apple,2025-05-03,10,0.588533
9,Apple,2025-05-04,7,0.533752


In [174]:
sentiment_df['avg_sentiment'] = sentiment_df['avg_sentiment'].round(3)

In [175]:
sentiment_df

,ticker,date,article_count,avg_sentiment
0,AMD,2025-05-03,8,0.617
1,AMD,2025-05-04,9,0.112
2,AMD,2025-05-05,9,0.194
3,AMD,2025-05-06,9,-0.152
4,Amazon,2025-05-03,10,0.510
5,Amazon,2025-05-04,10,0.591
6,Amazon,2025-05-05,10,0.713
7,Amazon,2025-05-06,10,0.869
8,Apple,2025-05-03,10,0.589
9,Apple,2025-05-04,7,0.534


In [177]:
sentiment_df.to_csv("assets/news_sentiment.csv", index = False)

In [19]:
news_df.isna().sum()

source          0
author         20
title           0
description    17
url             0
urlToImage     19
publishedAt     0
content         0
ticker          0
date            0
text            0
sentiment       0
dtype: int64

In [24]:
news_df[news_df['ticker'] == 'Netflix']# and news_df['date'] == '2025-05-03']

,source,author,title,description,url,urlToImage,publishedAt,content,ticker,date,text,sentiment
34,"{'id': 'techradar', 'name': 'TechRadar'}",Grace Dean,Nobody Wants This season 2: everything we know...,Nobody Wants This season 2 is currently in pro...,https://www.techradar.com/streaming/amazon-pri...,https://cdn.mos.cms.futurecdn.net/MsgiuN75JyTx...,2025-05-04T08:00:00Z,Nobody Wants This season 2: key information\r\...,Netflix,2025-05-04,Nobody Wants This season 2: everything we know...,0.997187
35,"{'id': None, 'name': 'Forbes'}","Erik Kain, Senior Contributor, \n Erik Kain, S...",One Of The Best Zombie Movies Ever Made Lands ...,A harrowing fight for survival aboard a train ...,https://www.forbes.com/sites/erikkain/2025/05/...,https://imageio.forbes.com/specials-images/ima...,2025-05-04T20:30:41Z,Train To Busan\r\nCredit: Next Entertainment W...,Netflix,2025-05-04,One Of The Best Zombie Movies Ever Made Lands ...,-0.998067
36,"{'id': None, 'name': 'Forbes'}","Paul Tassi, Senior Contributor, \n Paul Tassi,...",You Should Watch This Perfect 100% Scored Netf...,The best new show to hit Netflix in a while ha...,https://www.forbes.com/sites/paultassi/2025/05...,https://imageio.forbes.com/specials-images/ima...,2025-05-04T16:18:23Z,Netflix\r\nNetflix\r\nAt this point its well-e...,Netflix,2025-05-04,You Should Watch This Perfect 100% Scored Netf...,0.935252
37,"{'id': None, 'name': 'Biztoc.com'}",cnbc.com,Netflix executive says getting publicly fired ...,Bela Bajaria has had an incredible nine-year r...,https://biztoc.com/x/ae89246925013827,https://biztoc.com/cdn/ae89246925013827_s.webp,2025-05-04T14:22:56Z,Bela Bajaria has had an incredible nine-year r...,Netflix,2025-05-04,Netflix executive says getting publicly fired ...,-0.999807
38,"{'id': None, 'name': 'Screen Rant'}",Adrienne Tyler,"4 Years Before Pennywise, This Forgotten Netfl...",Bill Skarsgård is now best known for his horro...,https://screenrant.com/hemlock-grove-bill-skar...,https://static1.srcdn.com/wordpress/wp-content...,2025-05-04T13:20:17Z,Bill Skarsgård is now best known for his perfo...,Netflix,2025-05-04,"4 Years Before Pennywise, This Forgotten Netfl...",0.995962
83,"{'id': None, 'name': 'Giveawayoftheday.com'}",GOTD_Editor,StreamUlt Netflix Video Downloader 1.6.0,StreamUlt Netflix Video Downloader empowers us...,https://www.giveawayoftheday.com/streamult-net...,https://giveawayoftheday.com/wp-content/upload...,2025-05-03T08:00:00Z,StreamUlt Netflix Video Downloader empowers us...,Netflix,2025-05-03,StreamUlt Netflix Video Downloader 1.6.0 Strea...,-0.904976
84,"{'id': None, 'name': 'Forbes'}","Tim Lammers, Contributor, \n Tim Lammers, Cont...",Oscar Winner ‘I’m Still Here’ Gets Netflix Str...,“I’m Still Here” — the Best International Feat...,https://www.forbes.com/sites/timlammers/2025/0...,https://imageio.forbes.com/specials-images/ima...,2025-05-03T15:04:59Z,"Fernanda Torres in ""I'm Still Here.""\r\nSony P...",Netflix,2025-05-03,Oscar Winner ‘I’m Still Here’ Gets Netflix Str...,0.153722
85,"{'id': None, 'name': 'Forbes'}","Erik Kain, Senior Contributor, \n Erik Kain, S...",One Of The Best Zombie Movies Ever Made Lands ...,A harrowing fight for survival aboard a train ...,https://www.forbes.com/sites/erikkain/2025/05/...,https://imageio.forbes.com/specials-images/ima...,2025-05-03T16:29:19Z,Train To Busan\r\nCredit: Next Entertainment W...,Netflix,2025-05-03,One Of The Best Zombie Movies Ever Made Lands ...,-0.998067
86,"{'id': 'techradar', 'name': 'TechRadar'}",waynewilliams@onmail.com (Wayne Williams),Wi-Fi? More like Wow-Fi - researchers transmit...,Researchers demonstrate potential of optical w...,https://www.techradar.com/pro/wi-fi-more-like-...,https://cdn.mos.cms.futurecdn.net/YjrFTKj2ytUM...,2025-05-03T17:33:00Z,<ul><li>Researchers set new wireless data reco...,Netflix,2025-05-03,Wi-Fi? More like Wow-Fi - researchers transmit...,-0.999995
87,"{'id': None, 'name': 'CBM (Comic Book Movie)'}",MarkJulian,Netflix Drops The Sword! Rodriguez Reveals CON..